In [ ]:
from firedrake import *

comm = COMM_WORLD
mesh = UnitDiskMesh(3)

V = FunctionSpace(mesh, 'CG', 1)

def base(mesh, k):
    x, y = SpatialCoordinate(mesh)
    theta = atan2(x, y)
    cos_theta = cos(Constant(k)*theta)
    return cos_theta # + 1j*sin_theta

n = 10
M_inv = PETSc.Mat().createDense(size=(n, n), comm=comm)
M_np = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        M_np[i, j] = assemble(base(mesh, i)*base(mesh, j)*ds(domain=mesh))

M_inv_np = np.linalg.inv(M_np)
for i in range(n):
    for j in range(n):
        M_inv.setValue(i, j, M_inv_np[i, j])
M_inv.assemble()

v = TestFunction(V)
bs = []
for i in range(n):
    bs.append(
        assemble(base(mesh, i)*v*ds)
    )

with bs[0].dat.vec_ro as vec:
    print(vec.getSizes())

with bs[0].dat.vec_ro as vec:
    # ncl: local column size
    # gcl: global column size
    ncl, ncg = vec.getSizes()

bc = DirichletBC(V, 0, 'on_boundary')
bc_nodes = bc.nodes[bc.nodes < V.dof_dset.sizes[1]]
ncl = bc_nodes.size
B = PETSc.Mat().createDense(size=((n, n), (ncl, None)), comm=comm)
# B.setFromOptions()
# B.setPreallocationNNZ((5,5))
col_index = np.arange(ncl, dtype=np.int32)
for i in range(n):
    B.setValuesLocal(i, col_index, bs[i].dat.data[bc_nodes])

C = M_inv.ptap(B) # B^T M^{-1} B
PETSc.Sys.syncPrint(f"[{comm.rank}/{comm.size}] C size: {C.getSizes()}")

In [ ]:
vec.getLGMap()

In [ ]:
d = bs[0].dat

In [ ]:
u, v = TrialFunction(V), TestFunction(V)

In [ ]:
mat = assemble(u*v*dx)

In [ ]:
mat.M.sparsity.onnz, mat.M.sparsity.nnz